In [2]:
import os 
import sys
import numpy as np 
import pandas as pd
import EntropyHub as EH

In [13]:
sys.path.insert(0, '/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Scripts/Preprocessing')
from load_files import LoadFiles
from filter import NoiseFilter, HarmonicsFilter, remove_seizure_epochs
from constants import SYNGAP_baseline_start, SYNGAP_baseline_end, channel_variables, analysis_ls, SYNGAP_1_ls, SYNGAP_2_ls

In [14]:
class DispersionEntropy:
    
    def __init__(self, dat_array):
        self.dat_array = dat_array
        
        
    def disp_en(self):
        disp_en_ls =  []

        for epoch in self.dat_array:
            Dispx_1, Ppi_1 = EH.DispEn(epoch, m = 3, tau = 2, c = 4, Typex = 'ncdf')
            disp_en_ls.append(Dispx_1)
        
        disp_en_array = np.array(disp_en_ls)
        
        return disp_en_array

In [17]:
channel_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
channel_labels = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15]
directory_path = '/home/melissa/PREPROCESSING/SYNGAP1/numpyformat_baseline/'
results_path = '/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Results/Complexity/'

In [65]:
test = pd.read_csv(f'{results_path}{animal}_disp_entropy.csv')

In [66]:
test

,Epoch,Animal_ID,DispEn_0,DispEn_2,DispEn_3,DispEn_4,DispEn_5,DispEn_6,DispEn_7,DispEn_8,DispEn_9,DispEn_10,DispEn_11,DispEn_12,DispEn_13,DispEn_15
0,0,S7088,3.090650,3.058158,3.017531,3.059061,3.123320,3.223994,3.006113,3.308601,3.183524,2.720713,3.021759,3.040527,3.077740,3.120123
1,1,S7088,3.071096,3.097450,3.050144,3.023325,3.167386,3.306703,3.023722,3.220434,3.106109,2.562124,3.026563,3.051009,3.073493,2.989069
2,2,S7088,3.719661,3.613505,3.582491,3.681253,3.647630,3.768032,3.695178,3.774419,3.587188,2.109246,3.638558,3.596600,3.627428,3.648176
3,3,S7088,3.444537,3.370598,3.493466,3.503981,3.461554,3.487397,3.523646,3.495641,3.419008,2.614588,3.485518,3.454532,3.331004,3.435845
4,4,S7088,3.263770,3.098333,3.112725,3.215201,3.285292,3.332530,3.182181,3.250909,3.249975,2.626277,3.152589,3.086863,3.143514,3.141830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,17275,S7088,2.038076,2.029396,2.031133,2.029396,2.002477,2.003715,2.029396,2.002477,2.004933,2.005534,2.028214,2.029396,2.029396,2.027623
17276,17276,S7088,1.614086,1.604584,1.606485,1.600697,1.603813,1.588175,1.626143,1.541823,1.595815,1.549403,1.604519,1.610725,1.629896,1.624294
17277,17277,S7088,0.588220,0.588220,0.588220,0.588220,0.588220,0.588220,0.588220,0.588220,0.588220,0.588220,0.588220,0.588220,0.588220,0.588220
17278,17278,S7088,0.620481,0.620481,0.620481,0.620481,0.620481,0.620481,0.620481,0.620481,0.620481,0.620481,0.620481,0.620481,0.620481,0.620481


In [46]:
for animal in analysis_ls:
    print(f'loading {animal}')
    animal = str(animal)
    load_files = LoadFiles(directory_path, animal)
    if animal in SYNGAP_2_ls:
        data_1, data_2, brain_state_1, brain_state_2 = load_files.load_two_analysis_files(start_times_dict=SYNGAP_baseline_start, end_times_dict=SYNGAP_baseline_end)
        data = np.concatenate([data_1, data_2], axis = 1)
        noise_filter = NoiseFilter(data, brain_state_file = brain_state_1, channelvariables = channel_variables,ch_type = 'eeg')    
        bandpass_filtered_data = noise_filter.filter_data_type()
        channel_ls = []
        for chan_idx, chan_label in zip(channel_indices, channel_labels):
            print(chan_label)
            filter_channel = np.split(bandpass_filtered_data[chan_idx], 34560, axis = 0)
            disp_en_chan = DispersionEntropy(filter_channel)
            dispen_array = disp_en_chan.disp_en() 
            ent_df_chan = pd.DataFrame(data = {'Epoch': np.arange(0, 34560, 1), 'Animal_ID': [animal]*len(filter_channel),
                                             f'DispEn_{chan_label}': dispen_array})
            channel_ls.append(ent_df_chan)
            channel_concat = pd.concat(channel_ls) 
            channel_concat.to_csv(f'{results_path}{animal}_disp_entropy.csv')

loading S7088
loading S7092
loading S7086


In [53]:
filter_channel = np.split(bandpass_filtered_data[chan_idx], 34560, axis = 0)

In [57]:
len(filter_channel[0])

1252

In [44]:
for animal in analysis_ls:
    print(f'loading {animal}')
    animal = str(animal)
    load_files = LoadFiles(directory_path, animal)
    if animal in SYNGAP_2_ls:
        data_1, data_2, brain_state_1, brain_state_2 = load_files.load_two_analysis_files(start_times_dict=SYNGAP_baseline_start, end_times_dict=SYNGAP_baseline_end)
        data = np.concatenate([data_1, data_2], axis = 1)
        for data, brain_state in data_list:
            noise_filter = NoiseFilter(data, brain_state_file = brain_state, channelvariables = channel_variables,ch_type = 'eeg')    
            bandpass_filtered_data = noise_filter.filter_data_type()
            channel_ls = []
            for chan_idx, chan_label in zip(channel_indices, channel_labels):
                print(chan_label)
                filter_channel = np.split(bandpass_filtered_data[chan_idx], 17280, axis = 0)
                disp_en_chan = DispersionEntropy(filter_channel)
                dispen_array = disp_en_chan.disp_en() 
                ent_df_chan = pd.DataFrame(data = {'Epoch': np.arange(0, 17280, 1), 'Animal_ID': [animal]*len(filter_channel),
                                             f'DispEn_{chan_label}': dispen_array})
                channel_ls.append(ent_df_chan)
            channel_concat = pd.concat(channel_ls) 
            channel_concat.to_csv(f'{results_path}{animal}_disp_entropy.csv')
    elif animal in SYNGAP_1_ls:
        data, brain_state = load_files.load_one_analysis_file(start_times_dict=SYNGAP_baseline_start, end_times_dict=SYNGAP_baseline_end)
        data_list = [(data, brain_state)]
        for data, brain_state in data_list:
            noise_filter = NoiseFilter(data, brain_state_file = brain_state, channelvariables = channel_variables,ch_type = 'eeg')    
            bandpass_filtered_data = noise_filter.filter_data_type()
            channel_ls = []
            for chan_idx, chan_label in zip(channel_indices, channel_labels):
                print(chan_label)
                filter_channel = np.split(bandpass_filtered_data[chan_idx], 17280, axis = 0)
                disp_en_chan = DispersionEntropy(filter_channel)
                dispen_array = disp_en_chan.disp_en() 
                ent_df_chan = pd.DataFrame(data = {'Epoch': np.arange(0, 17280, 1), 'Animal_ID': [animal]*len(filter_channel),
                                             f'DispEn_{chan_label}': dispen_array})
                channel_ls.append(ent_df_chan)
            channel_concat = pd.concat(channel_ls) 
            channel_concat.to_csv(f'{results_path}{animal}_disp_entropy.csv')

loading S7088


KeyboardInterrupt: 

In [45]:
br

,brainstate,start_epoch,end_epoch
0,0,0,5
1,0,5,10
2,6,10,15
3,0,15,20
4,1,20,25
...,...,...,...
17275,7,86375,86380
17276,7,86380,86385
17277,7,86385,86390
17278,7,86390,86395


In [34]:
dispen_array = pd.concat(all_animals_ls)

In [35]:
all_animals

,Epoch,Animal_ID,DispEn_0,DispEn_2,DispEn_3,DispEn_4,DispEn_5,DispEn_6,DispEn_7,DispEn_8,DispEn_9,DispEn_10,DispEn_11,DispEn_12,DispEn_13,DispEn_15
0,0,S7088,-1.669286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,S7088,-0.156550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,S7088,0.028467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,S7088,-0.009136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,S7088,-0.612592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,17275,S7076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.080940
17276,17276,S7076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.417071
17277,17277,S7076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.305263
17278,17278,S7076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.461474
